In [1]:
import pandas as pd
import copy
from fuzzywuzzy import fuzz
from pandasql import sqldf
pysqldf = lambda q:sqldf(q,locals())
global un_df
from functools import partial

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, locals())

### Load the Data

In [2]:
df = pd.read_csv('UPDATED_SNA_data.csv',encoding = "ISO-8859-1",dtype={'IncidentNumber':object})

### Column Names

In [19]:
df.head(2)

,IncidentNumber,Agency,Identifier,IncidentDate,FirstName,MiddleName,LastName,FullName,DOB,DLNumber,IncidentLocationStreetAddress,IncidentCity,CrimeType,InvolvementType
0,1703327,AUBPD,AUBPD1703327,7/27/2017,Crystal,L,Schattenkerk,Crystal L Schattenkerk,4/26/1988,SCHATCL125J6,NaN,Auburn,NaN,Suspect
1,1706263,AUBPD,AUBPD1706263,5/22/2017,Patricia,NaN,Santos-Saunders,Patricia Santos-Saunders,3/6/1963,NaN,NaN,Auburn,MENT,Suspect


In [3]:
groups = df.groupby(['FullName','DOB'])

In [4]:
un_df = pd.DataFrame(list(groups.groups.keys()), columns = ['FullName', 'DOB'])

In [5]:
q1 = 'SELECT DISTINCT * FROM un_df'
ndf1 = sqldf(q1, globals())

In [6]:
ndf1 = ndf1.sort_values(['FullName'], ascending = True)

In [7]:
def get_ratio(row, target_name):
    name = row['FullName']
    return fuzz.token_sort_ratio(name, target_name)

#un_df['score'] = un_df.apply(get_ratio, axis=1)

In [8]:
def delete_name (row, target, DOB, un_df, MergeName):
    if target == 'None':
        return 
    index = row.name
    inner_index = un_df.index.get_loc(index)
    
    if inner_index - 30 < 0:
        Begin = 0
    else:
        Begin = inner_index - 30
    
    if inner_index + 30 > len(un_df)-1:
        End = len(un_df)-1
    else: 
        End = inner_index + 30
    
    un_df= un_df.iloc[Begin:End+1,]
    un_df2 = un_df[un_df['DOB'] == DOB]
    if un_df2.empty == False:
        #score = un_df1.apply(lambda x: get_ratio(x, target), axis = 1)
        un_df2['score'] = un_df2.apply(lambda x: get_ratio(x, target), axis = 1)
        un_df3 = un_df2[un_df2['score'] >= 80]

        un_df3 = un_df3.sort_values('FullName', ascending = False)
        
    
        if un_df3.empty == False:
           # print(un_df3)
            new_name = un_df3.iloc[0,0]
            # Get the index for the merged names
            merged_names = un_df3.index.tolist()
            MergeName[merged_names] = new_name

            #print(un_df.iloc[merged_names,])
            un_df.loc[merged_names, 'CheckName'] = 'None'
        
    
   # un_df1.drop(merged_names)

### Choose the Matching Range

In [9]:
un_df1 = ndf1.tail(200)
FullNames = un_df1['FullName']
un_df1['OriginName'] = un_df1['FullName']
un_df1['CheckName'] = un_df1['FullName']
MergeName = FullNames

C:\Users\Talino\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Talino\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
un_df1.apply(lambda x: delete_name(x, x['CheckName'], x['DOB'],un_df1,MergeName), axis = 1)

C:\Users\Talino\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Talino\Anaconda3\lib\site-packages\pandas\core\series.py:809: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\Talino\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\T

327721    None
342026    None
405803    None
30053     None
322618    None
430974    None
178618    None
221652    None
10002     None
327722    None
45953     None
59573     None
339591    None
381049    None
322619    None
381050    None
322620    None
265562    None
132091    None
137030    None
169529    None
63160     None
324035    None
322621    None
430976    None
430975    None
90375     None
322622    None
430978    None
430977    None
          ... 
13792     None
263609    None
77118     None
384658    None
164889    None
322641    None
45957     None
223566    None
182328    None
136422    None
47219     None
220754    None
387114    None
5504      None
5505      None
93585     None
154261    None
178619    None
405265    None
5506      None
405266    None
144636    None
32804     None
138768    None
154262    None
388175    None
434632    None
384659    None
173614    None
144637    None
Length: 200, dtype: object

In [21]:
un_df1 = un_df1.sort_values(['FullName'])

### Displaying Merged Names

In [12]:
q = 'SELECT * FROM un_df1 WHERE FullName in (SELECT FullName FROM un_df1 GROUP BY FullName HAVING COUNT(*) > 1)'

ndf = sqldf(q, globals())

In [13]:
ndf

,FullName,DOB,OriginName,CheckName
0,Zohra Rahgozar,None,Zohra Rahgozar,Zohra Rahgozar
1,Zohra Rahgozar,6/24/1963,Zohra Rahgozar,None
2,Zoila Castillo-Hernandez,6/27/1972,Zoila Castillo Hernandez,None
3,Zoila Castillo-Hernandez,6/27/1972,Zoila Castillo-Hernandez,None
4,Zolita I Barron,4/24/1993,Zoila I Barron,None
5,Zoilio Antonio Velesco,6/26/1968,Zoilio Antonio Velasco,None
6,Zoilio Antonio Velesco,6/26/1968,Zoilio Antonio Velesco,None
7,Zola E Lewis,12/13/1989,Zola Lewis,None
8,Zola E Lewis,None,Zola E Lewis,Zola E Lewis
9,Zola E Lewis,12/19/1989,Zola E Lewis,None


### Match the name in the original File

In [14]:
mydict = dict(zip(ndf.OriginName, ndf.FullName))

In [16]:
df["FullName"].replace(mydict, inplace=True)

In [17]:
df[df.FullName == 'Zynekwa Shantasia Williams']

,IncidentNumber,Agency,Identifier,IncidentDate,FirstName,MiddleName,LastName,FullName,DOB,DLNumber,IncidentLocationStreetAddress,IncidentCity,CrimeType,InvolvementType
51522,201677115,SPD,SPD201677115,3/4/2016,Zynekwa,S,Williams,Zynekwa Shantasia Williams,11/15/1993,WILLIZS073QN,4800 SAND POINT WY NE,Seattle,Child-Other,Suspect
51523,201677115,SPD,SPD201677115,3/4/2016,Zynekwa,S,Williams,Zynekwa Shantasia Williams,11/15/1993,WILLIZS073QN,4800 SAND POINT WY NE,Seattle,Child-Other,Suspect
114540,2016143300,SPD,SPD2016143300,4/25/2016,Zynekwa,S,Williams,Zynekwa Shantasia Williams,11/15/1993,WILLIZS073QN,5700 BLOCK M L KING JR WY S,Seattle,Collision - Traffic,Suspect
246533,160003324,FWPD,FWPD160003324,3/4/2016,Zynekwa,Shantasia,Williams,Zynekwa Shantasia Williams,11/15/1993,WILLIZS073QN,27606 Pacific Hwy S #K102,Federal Way,Assault 2,Suspect
258559,1707460,AUBPD,AUBPD1707460,6/18/2017,Zynekwa,NaN,Williams,Zynekwa Shantasia Williams,11/15/1993,NaN,NaN,Auburn,TOFF,Suspect
